# Quantized Deployment and Throughput/Accuracy Testing
This notebook deploys the quantized model on an FPGA and performs throughput and accuracy testing.
If the desired deployment is on a PYNQ board, this notebook should be run from the PYNQ host.

In [1]:
from output_final.driver.finn.core.datatype import DataType
from driver_base import FINNExampleOverlay
import numpy as np
import proc_for_infer as pfi
from utils_pynq import load_split_np_data

In [2]:
#set up model info
# dictionary describing the I/O of the FINN-generated accelerator
io_shape_dict = {
    # FINN DataType for input and output tensors
    "idt" : DataType.INT6,
    "odt" : DataType.INT6,
    # shapes for input and output tensors (NHWC layout)
    "ishape_normal" : (1, 14),
    "oshape_normal" : (1, 1),
    # folded / packed shapes below depend on idt/odt and input/output
    # PE/SIMD parallelization settings -- these are calculated by the
    # FINN compiler.
    "ishape_folded" : (1, 1, 14),
    "oshape_folded" : (1, 1, 1),
    "ishape_packed" : (1, 1, 11),
    "oshape_packed" : (1, 1, 1)
}
platform = "zynq-iodma"
bitfile = "output_final/deploy/bitfile/finn-accel.bit"
input_file = "data/X_train.npy"
output_comparison = "data/Y_train.npy"
runtime_weights = "output_final/deploy/driver/runtime_weights/"
batch_size = 1

In [3]:
#create accelerator
accel = FINNExampleOverlay(bitfile_name=bitfile, platform=platform, 
                          io_shape_dict=io_shape_dict, batch_size=batch_size, fclk_mhz=4,
                          runtime_weight_dir=runtime_weights)

### Throughput Test
Run a basic throughput test and print the results

In [6]:
res = accel.throughput_test()
print(res)

{'runtime[ms]': 0.4906654357910156, 'throughput[images/s]': 2038.0485908649173, 'DRAM_in_bandwidth[Mb/s]': 0.02241853449951409, 'DRAM_out_bandwidth[Mb/s]': 0.0020380485908649171, 'fclk[mhz]': 3.99996, 'batch_size': 1, 'fold_input[ms]': 5.3882598876953125e-05, 'pack_input[ms]': 0.008006811141967773, 'copy_input_data_to_device[ms]': 0.001003265380859375, 'copy_output_data_from_device[ms]': 0.0002522468566894531, 'unpack_output[ms]': 0.0009839534759521484, 'unfold_output[ms]': 2.7418136596679688e-05}


### Accuracy Test
Perform inference on testing dataset and compare to actual values.

In [7]:
#accel.batch_size(229538)
print(accel.ishape_normal)

(1, 14)


In [4]:
datasets = load_split_np_data()
print(datasets)

X_train shape: (229538, 14)
X_val shape: (12752, 14)
X_test shape: (12752, 14)
Y_train shape: (229538, 1)
Y_val shape: (12752, 1)
Y_test shape: (12752, 1)
Using saved split data
[array([[  5.80000000e+01,   2.00000000e+00,   3.69370341e+00, ...,
          1.64524792e+03,   1.56832642e+03,   9.73000366e+02],
       [  7.30000000e+01,   3.00000000e+00,   1.63631487e+00, ...,
          1.09457141e+03,   8.94265137e+02,   7.37251587e+02],
       [  7.40000000e+01,   2.00000000e+00,   1.15914593e+01, ...,
          4.53916113e+03,   1.46776880e+03,   7.66278015e+02],
       ..., 
       [  6.40000000e+01,   2.00000000e+00,   9.21862221e+00, ...,
          1.73535828e+03,   1.19223145e+03,   6.71937500e+02],
       [  7.10000000e+01,   2.00000000e+00,   7.65338302e-01, ...,
          8.91962585e+02,   1.62368567e+03,   8.88209412e+02],
       [  9.10000000e+01,   2.00000000e+00,   8.17640972e+00, ...,
          2.95410669e+03,   1.74744006e+03,   8.36169189e+02]]), array([[  5.70000000e+01, 

In [5]:
inps = datasets[1]
exp_out = datasets[4]
valid_size = len(exp_out)
num_batches = int(valid_size/batch_size)
running_error_square = 0
for i in range(0, num_batches):
    batch = inps[(i*batch_size):((i+1)*batch_size)]
    batch_exp_out = exp_out[(i*batch_size):((i+1)*batch_size)]
    #print(batch.shape)
    proc_batch = pfi.preproc(batch)
    #proc_batch = np.flip(proc_batch, axis=1)
    batch_out = accel.execute(proc_batch)
    batch_proc_out = pfi.postproc(batch_out)
    batch_errs = batch_proc_out-batch_exp_out
    print("exp: " + str(batch_exp_out) + " act: " + str(batch_proc_out) + " preproc: " + str(batch_out))
    batch_sq_errs = batch_errs*batch_errs
    running_error_square += sum(batch_sq_errs)
    #batch = valid[(i*batch_size):((i+1)*batch_size)]
    #inp = batch[0]
    #inp = FACILE_preproc(inp.float())
    #inp = inp.numpy()
    #inp_dict = {in_tensor : inp}
    #exp_out = batch[1].numpy()
    #out_dict = exe_onnx(model, inp_dict)
    #out = out_dict[out_tensor]
    #out = tensor(out,dtype=float32)
    #out = FACILE_postproc(out)
    #print(out)
print("MSE: " + str(running_error_square / (num_batches * batch_size)))

exp: [[ 132.65643311]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 845.6675]] preproc: [[ 25.]]
exp: [[ 0.]] act: [[ 439.7471]] preproc: [[ 13.]]
exp: [[ 21.67700386]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 0.]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 95.71929932]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 0.]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 35.5912323]] act: [[ 304.4403]] preproc: [[ 9.]]
exp: [[ 42.45626068]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 25.850317]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 126.25019836]] act: [[ 405.9204]] preproc: [[ 12.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 27.163414]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 79.58286285]] act: [[ 236.7869]] preproc: [[ 7.]]
exp: [[ 0.]] act: [[ 575.05

exp: [[ 0.]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 0.]] act: [[ 642.7073]] preproc: [[ 19.]]
exp: [[ 32.41735077]] act: [[ 439.7471]] preproc: [[ 13.]]
exp: [[ 0.]] act: [[ 575.0539]] preproc: [[ 17.]]
exp: [[ 39.0557785]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 24.95316696]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 27.62182808]] act: [[ 575.0539]] preproc: [[ 17.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 372.0937]] preproc: [[ 11.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 29.22259521]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 70.69464111]] act: [[ 575.0539]] preproc: [[ 17.]]
exp: [[ 45.79159164]] act: [[ 405.9204]] preproc: [[ 12.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 54.07184601]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 0.]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 22.73404312]] act: [[ 507

exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 0.]] act: [[ 473.5738]] preproc: [[ 14.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 50.04506683]] act: [[ 710.3607]] preproc: [[ 21.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 27.3862381]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 0.]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 94.57099915]] act: [[ 473.5738]] preproc: [[ 14.]]
exp: [[ 36.98365784]] act: [[ 439.7471]] preproc: [[ 13.]]
exp: [[ 21.82725143]] act: [[ 642.7073]] preproc: [[ 19.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 744.1874]] preproc: [[ 22.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 473.5738]] preproc: [[ 14.]]
exp: [[ 8.85501194]] act: [[ 473.5738]] preproc: [[ 14.]]
exp: [[ 13.55456638]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 28.29098701]] act: [[ 338.267]] preproc: [[ 10.]]
exp: [[ 27.86312675]] act: [[ 37

exp: [[ 106.57025909]] act: [[ 169.1335]] preproc: [[ 5.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 22.47159958]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 0.]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 44.75529861]] act: [[ 575.0539]] preproc: [[ 17.]]
exp: [[ 31.04143715]] act: [[ 405.9204]] preproc: [[ 12.]]
exp: [[ 25.21374512]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 52.44209671]] act: [[ 642.7073]] preproc: [[ 19.]]
exp: [[ 0.]] act: [[ 642.7073]] preproc: [[ 19.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 25.19186783]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 86.88653564]] act: [[ 202.9602]] preproc: [[ 6.]]
exp: [[ 0.]] act: [[ 338.267]] preproc: [[ 10.]]
exp: [[ 184.16322327]] act: [[ 270.6136]] preproc: [[ 8.]]
exp: [[ 29.46250725]] ac

exp: [[ 0.]] act: [[ 710.3607]] preproc: [[ 21.]]
exp: [[ 0.]] act: [[ 405.9204]] preproc: [[ 12.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 39.53383255]] act: [[ 642.7073]] preproc: [[ 19.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 30.53726578]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 29.73623276]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 0.]] act: [[ 473.5738]] preproc: [[ 14.]]
exp: [[ 40.41942215]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 0.]] act: [[ 473.5738]] preproc: [[ 14.]]
exp: [[ 16.92591286]] act: [[ 439.7471]] preproc: [[ 13.]]
exp: [[ 0.]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 0.]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 0.]] act: [[ 575.0539]] preproc: [[ 17.]]
exp: [[ 11.26241493]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 53.56375885]] act: [[ 135.3068]] preproc: [[ 4.]]
exp: [[ 0.]] act: [[ 642.7073]] preproc: [[ 19.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: 

...

exp: [[ 0.]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 108.90071106]] act: [[ 879.4942]] preproc: [[ 26.]]
exp: [[ 79.24585724]] act: [[ 710.3607]] preproc: [[ 21.]]
exp: [[ 304.96987915]] act: [[ 744.1874]] preproc: [[ 22.]]
exp: [[ 0.]] act: [[ 642.7073]] preproc: [[ 19.]]
exp: [[ 178.70335388]] act: [[ 439.7471]] preproc: [[ 13.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 39.39519119]] act: [[ 338.267]] preproc: [[ 10.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 166.67944336]] act: [[ 405.9204]] preproc: [[ 12.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 58.87995911]] act: [[ 202.9602]] preproc: [[ 6.]]
exp: [[ 67.01494598]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 24.84449196]] act: [[ 405.9204]] preproc: [[ 12.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 30.06176376]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 0.]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 22.

exp: [[ 0.]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 49.15772629]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 0.]] act: [[ 473.5738]] preproc: [[ 14.]]
exp: [[ 0.]] act: [[ 744.1874]] preproc: [[ 22.]]
exp: [[ 55.90203857]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 22.6722374]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 53.35802078]] act: [[ 101.4801]] preproc: [[ 3.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 29.23820496]] act: [[ 372.0937]] preproc: [[ 11.]]
exp: [[ 0.]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 0.]] act: [[ 405.9204]] preproc: [[ 12.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 0.]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 33.48374176]] act: [[ 338.267]] preproc: [[ 10.]]
exp: [[ 40.09639359]] act: [[ 744.1874]] preproc: [[ 22.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: 

exp: [[ 0.]] act: [[ 575.0539]] preproc: [[ 17.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 34.31891632]] act: [[ 338.267]] preproc: [[ 10.]]
exp: [[ 25.18685532]] act: [[ 473.5738]] preproc: [[ 14.]]
exp: [[ 20.33363724]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 44.16918945]] act: [[ 642.7073]] preproc: [[ 19.]]
exp: [[ 50.87004089]] act: [[ 67.6534]] preproc: [[ 2.]]
exp: [[ 33.46318817]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 33.10508728]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 86.77719879]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 34.65787506]] act: [[ 372.0937]] preproc: [[ 11.]]
exp: [[ 29.60894966]] act: [[ 575.0539]] preproc: [[ 17.]]
exp: [[ 0.]] act: [[ 473.5738]] preproc: [[ 14.]]
exp: [[ 41.21383667]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 19.8972187]] act: [[ 642.7073]] preproc: [[ 19.]]
exp: [[ 31.48828888]] act: [[ 405.9204]] preproc:

exp: [[ 0.]] act: [[ 439.7471]] preproc: [[ 13.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 31.85860634]] act: [[ 710.3607]] preproc: [[ 21.]]
exp: [[ 35.31397247]] act: [[ 405.9204]] preproc: [[ 12.]]
exp: [[ 22.09577942]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 39.09338379]] act: [[ 338.267]] preproc: [[ 10.]]
exp: [[ 0.]] act: [[ 473.5738]] preproc: [[ 14.]]
exp: [[ 29.25230408]] act: [[ 304.4403]] preproc: [[ 9.]]
exp: [[ 73.97322845]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 75.64120483]] act: [[ 473.5738]] preproc: [[ 14.]]
exp: [[ 0.]] act: [[ 642.7073]] preproc: [[ 19.]]
exp: [[ 0.]] act: [[ 710.3607]] preproc: [[ 21.]]
exp: [[ 0.]] act: [[ 575.0539]] preproc: [[ 17.]]
exp: [[ 0.]] act: [[ 439.7471]] preproc: [[ 13.]]
exp: [[ 38.40453339]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 0.]] act: [[ 608.8806]] preproc: [[ 18.]]
exp: [[ 145.61097717]] act: [[ 236.7869]] preproc: [[ 7.]]
exp: [[ 0.]] act: [[ 50

exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 575.0539]] preproc: [[ 17.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 676.534]] preproc: [[ 20.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 122.32891846]] act: [[ 710.3607]] preproc: [[ 21.]]
exp: [[ 0.]] act: [[ 473.5738]] preproc: [[ 14.]]
exp: [[ 0.]] act: [[ 575.0539]] preproc: [[ 17.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 17.68338013]] act: [[ 541.2272]] preproc: [[ 16.]]
exp: [[ 36.00334549]] act: [[ 439.7471]] preproc: [[ 13.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 22.7145462]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 507.4005]] preproc: [[ 15.]]
exp: [[ 0.]] act: [[ 642.7073]] preproc: [[ 19.]]
exp: [[ 23.09434